In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ls.config.loader import load_config
import pandas as pd
import os
import numpy as np

### Load configuration file

In [3]:
cfg = load_config("../configs/config.yaml")

In [4]:
df = pd.read_csv(os.path.join(cfg.dataset.data_folder, "icbhi_metadata.csv"))

### Create columns for 4-class and 2-class problems and cycle duraitons

In [5]:
df['label_4c'] = df.apply(lambda r: 
    'Crackle' if r.Crackles==1 and r.Wheezes==0 else
    'Wheeze'  if r.Crackles==0 and r.Wheezes==1 else
    'Both'    if r.Crackles==1 and r.Wheezes==1 else
    'Normal', axis=1)

In [6]:
df["label_2c"] = np.where((df["Crackles"] + df["Wheezes"]) > 0, "Abnormal", "Normal")
df['CycleDur'] = df['CycleEnd'] - df['CycleStart']

df.head()

,PID,Filename,CycleIndex,CycleStart,CycleEnd,Crackles,Wheezes,Split,Device,Fold,Age,Sex,BMI,CW,CH,Disease,AuscLoc,label_4c,label_2c,CycleDur
0,101,101_1b1_Al_sc_Meditron,0,0.036,0.579,0,0,test,Meditron,1,3.0,F,NaN,19.0,99.0,URTI,Al,Normal,Normal,0.543
1,101,101_1b1_Al_sc_Meditron,1,0.579,2.450,0,0,test,Meditron,1,3.0,F,NaN,19.0,99.0,URTI,Al,Normal,Normal,1.871
2,101,101_1b1_Al_sc_Meditron,2,2.450,3.893,0,0,test,Meditron,1,3.0,F,NaN,19.0,99.0,URTI,Al,Normal,Normal,1.443
3,101,101_1b1_Al_sc_Meditron,3,3.893,5.793,0,0,test,Meditron,1,3.0,F,NaN,19.0,99.0,URTI,Al,Normal,Normal,1.900
4,101,101_1b1_Al_sc_Meditron,4,5.793,7.521,0,0,test,Meditron,1,3.0,F,NaN,19.0,99.0,URTI,Al,Normal,Normal,1.728


### Check for patient leakage/overlap between train and test sets

In [7]:
# Basic integrity checks
assert df["PID"].nunique() == df.groupby("PID")["Split"].nunique().max(), "A PID appears in both splits!"

AssertionError: A PID appears in both splits!

In [8]:
dup_pids = (
    df.groupby("PID")["Split"]
      .nunique()
      .reset_index()
      .query("Split > 1")
)

if not dup_pids.empty:
    print("Patients appearing in both training and test splits:")
    display(df[df["PID"].isin(dup_pids["PID"])][["PID","Filename", "Split"]].drop_duplicates())

Patients appearing in both training and test splits:


,PID,Filename,Split
2896,156,156_2b3_Al_mc_AKGC417L,test
2902,156,156_2b3_Ar_mc_AKGC417L,train
2908,156,156_2b3_Ll_mc_AKGC417L,train
2914,156,156_2b3_Lr_mc_AKGC417L,test
2920,156,156_2b3_Pl_mc_AKGC417L,test
2926,156,156_2b3_Pr_mc_AKGC417L,train
2932,156,156_5b3_Al_mc_AKGC417L,train
2941,156,156_5b3_Ar_mc_AKGC417L,test
2950,156,156_5b3_Ll_mc_AKGC417L,test
2959,156,156_5b3_Lr_mc_AKGC417L,train


In [9]:
assert (df["CycleDur"] > 0).all(), "Non-positive cycle duration found"

### Metadata Dataset Exploitation and Descriptive Statistics

In [10]:
def print_section(title, df):
    print("\n" + "="*80)
    print(f"{title.upper()}")
    print("="*80)
    print(df.to_string())
    print()

## Class imbalances Calculation or the official fixed train/test split (2-class & 4-class problem)

### Global Dataset (Train + Test)

In [11]:
print_section("4-Class Global Counts", df["label_4c"].value_counts().to_frame("count"))
print_section("4-Class Global Proportions", df["label_4c"].value_counts(normalize=True).round(3).to_frame("proportion"))

print_section("2-Class Global Counts", df["label_2c"].value_counts().to_frame("count"))
print_section("2-Class Global Proportions", df["label_2c"].value_counts(normalize=True).round(3).to_frame("proportion"))


4-CLASS GLOBAL COUNTS
          count
label_4c       
Normal     3642
Crackle    1864
Wheeze      886
Both        506


4-CLASS GLOBAL PROPORTIONS
          proportion
label_4c            
Normal         0.528
Crackle        0.270
Wheeze         0.128
Both           0.073


2-CLASS GLOBAL COUNTS
          count
label_2c       
Normal     3642
Abnormal   3256


2-CLASS GLOBAL PROPORTIONS
          proportion
label_2c            
Normal         0.528
Abnormal       0.472



### Class imbalances for each split (4-class problem)

In [12]:
# By split
by_split_4c = df.pivot_table(index="Split", columns="label_4c", values="Filename", aggfunc="count").fillna(0).astype(int)
by_split_prop_4c = by_split_4c.div(by_split_4c.sum(axis=1), axis=0).round(3)
print_section("4-Class Counts by Split", by_split_4c)
print_section("4-Class Proportions by Split", by_split_prop_4c)
# print("\n4c counts by Split:\n", by_split)
# print("\n4c proportions by Split:\n", by_split_prop)


4-CLASS COUNTS BY SPLIT
label_4c  Both  Crackle  Normal  Wheeze
Split                                  
test       143      649    1579     385
train      363     1215    2063     501


4-CLASS PROPORTIONS BY SPLIT
label_4c   Both  Crackle  Normal  Wheeze
Split                                   
test      0.052    0.235   0.573   0.140
train     0.088    0.293   0.498   0.121



### Class imbalances for each split (2-class problem)

In [13]:
# By split
by_split = df.pivot_table(index="Split", columns="label_2c", values="Filename", aggfunc="count").fillna(0).astype(int)
by_split_prop = by_split.div(by_split.sum(axis=1), axis=0).round(3)
print_section("2-Class Counts by Split", by_split)
print_section("2-Class Proportions by Split", by_split_prop)


2-CLASS COUNTS BY SPLIT
label_2c  Abnormal  Normal
Split                     
test          1177    1579
train         2079    2063


2-CLASS PROPORTIONS BY SPLIT
label_2c  Abnormal  Normal
Split                     
test         0.427   0.573
train        0.502   0.498



### Findings:

* Global 4-class: Normal 52.8%, Crackle 27.0%, Wheeze 12.8%, Both 7.3%.
* 2-class: Normal 52.8% vs Abnormal 47.2% → close to balanced.

Train–test label mix isn’t identical

* Test has more Normal (57.3%) and fewer Crackle (23.5%) & Both (5.2%) than Train (Normal 49.8%, Crackle 29.3%, Both 8.8%).   
 -> Expect lower sensitivity on test (especially for Crackle/Both) if you tune on train-like prevalence.

## Descriptive statistics for the breathing cycle duraiton for the 4-class problem

In [15]:
dur_stats = df.groupby("label_4c")["CycleDur"].describe().round(3)
# print("\nCycle duration stats (sec) by 4c:\n", dur_stats)

# per-split duration sanity
dur_split = df.groupby(["Split","label_4c"])["CycleDur"].describe().round(3)
# print("\nCycle duration stats by Split × 4c:\n", dur_split)

print_section("Cycle Duration Stats (sec) by 4-Class Label", dur_stats)
print_section("Cycle Duration Stats (sec) by Split × Label", dur_split)


CYCLE DURATION STATS (SEC) BY 4-CLASS LABEL
           count   mean    std    min    25%    50%    75%     max
label_4c                                                          
Both       506.0  3.060  1.092  0.571  2.238  2.904  3.558   8.592
Crackle   1864.0  2.785  0.952  0.367  2.137  2.629  3.449   8.736
Normal    3642.0  2.607  1.275  0.200  1.716  2.443  3.288  16.163
Wheeze     886.0  2.703  1.143  0.228  1.803  2.584  3.357   9.217


CYCLE DURATION STATS (SEC) BY SPLIT × LABEL
                 count   mean    std    min    25%    50%    75%     max
Split label_4c                                                          
test  Both       143.0  3.358  1.316  1.057  2.277  3.233  4.007   8.592
      Crackle    649.0  3.093  0.923  0.443  2.605  3.212  3.773   6.411
      Normal    1579.0  2.391  1.037  0.286  1.564  2.310  3.141   7.632
      Wheeze     385.0  2.668  1.259  0.228  1.583  2.512  3.456   9.217
train Both       363.0  2.942  0.967  0.571  2.238  2.734  3.417   7.

## Q1. Do breathing cycle durations differ across sound labels?

H₀:

Cycle duration distributions are identical across Normal, Crackle, Wheeze, and Both.

H₁:

At least one label’s duration distribution differs.

Interpretation phrasing for the paper

To ensure independence across samples, we performed stratified patient-level sampling: for each sound class, one breathing cycle was randomly selected per patient exhibiting that class. This process was repeated 100 times (bootstrap resampling) to account for selection variance. Normality and variance homogeneity were violated (Shapiro–Wilk and Levene’s p<0.001), thus a Kruskal–Wallis test was applied on each bootstrap replicate.

### Goal

### Quantify whether the breathing-cycle duration differs systematically between respiratory sound categories (Normal, Crackle, Wheeze, Both).
### This assesses whether cycle duration carries discriminative information about adventitious sounds, or if it is merely random variability.

* Challenges  
	•	Each patient contributes multiple cycles → dependent samples.  
	•	Distributions are non-normal and heteroscedastic.  
	•	The dataset is imbalanced across labels.  

* Approach.  
	1.	Patient-wise bootstrap resampling — one random cycle per label per patient, repeated N times, ensures approximate independence.    
	2.	Distributional testing — for each resample:   
	•	Shapiro–Wilk for normality, Levene for equal variances.   
	•	Choose parametric (ANOVA/Welch) or non-parametric (Kruskal–Wallis).  
	3.	Effect quantification — compute ε² (variance explained) and, if appropriate, pairwise Mann–Whitney tests with Cliff’s δ.  
	4.	Stability assessment — summarize p-values, significance fractions, and effect sizes across bootstraps.    
	5.	Repeat for both label_4c (4-class) and label_2c (Normal vs Abnormal) to compare sensitivity.  
	

In [16]:
import statsmodels.formula.api as smf

model = smf.mixedlm("CycleDur ~ C(label_4c)", data=df, groups=df["PID"])
result = model.fit()
print(result.summary())

              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    CycleDur  
No. Observations:    6898       Method:                REML      
No. Groups:          126        Scale:                 0.7533    
Min. group size:     4          Log-Likelihood:        -9041.1718
Max. group size:     507        Converged:             Yes       
Mean group size:     54.7                                        
-----------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept               3.070    0.095 32.216 0.000  2.883  3.257
C(label_4c)[T.Crackle] -0.355    0.050 -7.084 0.000 -0.453 -0.257
C(label_4c)[T.Normal]  -0.436    0.050 -8.717 0.000 -0.533 -0.338
C(label_4c)[T.Wheeze]  -0.259    0.054 -4.836 0.000 -0.365 -0.154
Group Var               0.842    0.131                           



In [17]:
model = smf.mixedlm(
    "CycleDur ~ C(label_4c, Treatment(reference='Wheeze'))",
    data=df,
    groups=df["PID"]
)
result = model.fit()
print(result.summary())

                             Mixed Linear Model Regression Results
Model:                          MixedLM              Dependent Variable:              CycleDur  
No. Observations:               6898                 Method:                          REML      
No. Groups:                     126                  Scale:                           0.7533    
Min. group size:                4                    Log-Likelihood:                  -9041.1718
Max. group size:                507                  Converged:                       Yes       
Mean group size:                54.7                                                            
------------------------------------------------------------------------------------------------
                                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------------
Intercept                                              2.811

A linear mixed-effects model with patient ID as a random intercept (n = 6 898 cycles, 126 patients) revealed significant fixed effects of sound label on cycle duration (χ² = …, p < 0.001).
Relative to normal cycles (mean = 2.64 s), both-type cycles were +0.44 s (p < 0.001), crackles +0.08 s (p = 0.007), and wheezes +0.18 s (p < 0.001) longer.
The intra-class correlation (ICC = 0.53) indicated that roughly half the total variance was attributable to between-patient differences.
These results confirm, under proper modeling of repeated measures, that abnormal sounds are associated with modestly prolonged respiratory cycles—consistent with the non-parametric bootstrap analysis.

### Abnormality correlates with longer cycle duration, though the effect is small (~0.1–0.4 s) and highly patient-dependent.

In [ ]:
import pandas as pd
from scipy.stats import friedmanchisquare

# Prepare per-patient mean durations per label
df_pmeans = (
    df.groupby(["PID", "label_4c"])["CycleDur"]
      .mean()
      .unstack()  # wide format: one column per label
)

# Keep only patients with ≥2 labels
df_valid = df_pmeans.dropna() # dropna(thresh=2)

# Drop labels missing across all patients if needed
labels = [c for c in df_valid.columns if df_valid[c].notna().any()]
df_valid = df_valid[labels]

# Apply Friedman test (requires at least 3 columns)
if len(labels) >= 3:
    stat, p = friedmanchisquare(*[df_valid[l] for l in labels])
    print("=============================================")
    print(" Friedman test on per-patient mean durations ")
    print("=============================================")
    print(f"Patients included: {len(df_valid)}")
    print(f"Labels tested: {labels}")
    print(f"χ²({len(labels)-1}) = {stat:.3f}, p = {p:.5f}")
else:
    print("Not enough label categories per patient for Friedman test.")

 Friedman test on per-patient mean durations 
Patients included: 28
Labels tested: ['Both', 'Crackle', 'Normal', 'Wheeze']
χ²(3) = 9.643, p = 0.02186


In [22]:
from scipy.stats import wilcoxon

def paired_test(a, b):
    df_pair = df_pmeans[[a,b]].dropna()
    stat, p = wilcoxon(df_pair[a], df_pair[b])
    print(f"{a} vs {b}: n={len(df_pair)}, W={stat:.3f}, p={p:.5f}, {'reject H0' if p<0.05 else 'fail to reject H0'}")

paired_test("Normal","Crackle")
paired_test("Normal","Wheeze")
paired_test("Normal","Both")
paired_test("Crackle","Wheeze")
paired_test("Crackle","Both")
paired_test("Wheeze","Both")

Normal vs Crackle: n=72, W=814.000, p=0.00502, reject H0
Normal vs Wheeze: n=62, W=657.000, p=0.02509, reject H0
Normal vs Both: n=34, W=102.000, p=0.00050, reject H0
Crackle vs Wheeze: n=45, W=475.000, p=0.81545, fail to reject H0
Crackle vs Both: n=30, W=164.000, p=0.16418, fail to reject H0
Wheeze vs Both: n=34, W=148.000, p=0.00957, reject H0


“Paired Wilcoxon signed-rank tests across patients who exhibited multiple sound types revealed that abnormal cycles (Crackle, Wheeze, Both) were significantly longer than normal ones (all p < 0.03). No consistent differences were observed among abnormal categories. Thus, breathing-cycle duration carries moderate discriminative value for distinguishing normal vs abnormal sounds but provides limited information for differentiating specific adventitious types.”

In [23]:
import numpy as np, pandas as pd
from scipy.stats import shapiro, levene, f_oneway, kruskal, mannwhitneyu
import itertools, warnings
warnings.filterwarnings("ignore")

In [24]:
# ---------------------------------------------------------------
# Smart patient-wise stratified sampler (1 sample per patient)
# Prioritizes abnormal (minority) classes first
# ---------------------------------------------------------------
def stratified_patient_sampling_priority(df, label_col, seed=42):
    np.random.seed(seed)
    sampled = []
    patients_sampled = set()

    # Sort labels by ascending frequency → minority first
    for lbl in df[label_col].value_counts().sort_values().index:
        sub = df[df[label_col] == lbl]
        # sample one per patient, skipping already used
        sub = sub[~sub["PID"].isin(patients_sampled)]
        s = sub.groupby("PID", group_keys=False).apply(lambda x: x.sample(1, random_state=seed))
        sampled.append(s)
        patients_sampled.update(s["PID"].unique())

    return pd.concat(sampled).reset_index(drop=True)

In [25]:
# ---------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------
def stratified_patient_sampling(df, label_col, seed=42):
    np.random.seed(seed)
    parts = []
    for lbl, g in df.groupby(label_col):
        s = g.groupby("PID", group_keys=False).apply(lambda x: x.sample(1, random_state=seed))
        parts.append(s)
    return pd.concat(parts).reset_index(drop=True)

def kw_epsilon_squared(H, k, N):
    return np.nan if (N - k) <= 0 else (H - k + 1) / (N - k)

def cliffs_delta(x, y):
    nx, ny = len(x), len(y)
    diff = np.subtract.outer(x, y)
    delta = (np.sum(diff > 0) - np.sum(diff < 0)) / (nx * ny)
    return delta

# ---------------------------------------------------------------
# Adaptive test selector
# ---------------------------------------------------------------
def cycle_duration_test(df_samp, label_col):
    labels = df_samp[label_col].unique()
    groups = [df_samp.loc[df_samp[label_col]==l, "CycleDur"] for l in labels]
    
    # Normality check for each group/class
    shapiro_p = [shapiro(g)[1] for g in groups]
    # print(shapiro_p)
    normal = all(p > 0.05 for p in shapiro_p)
    # Check if equal variances
    levene_p = levene(*groups)[1]
    equal_var = levene_p > 0.05
    k, N = len(groups), sum(len(g) for g in groups)

    if normal and equal_var:
        test_type = "ANOVA"
        stat, p = f_oneway(*groups)
        eps2 = np.nan
    elif normal and not equal_var:
        test_type = "Welch_ANOVA"
        stat, p = f_oneway(*groups)  # approximate
        eps2 = np.nan
    else:
        test_type = "Kruskal_Wallis"
        stat, p = kruskal(*groups)
        eps2 = kw_epsilon_squared(stat, k, N)

    return dict(test=test_type, stat=stat, p=p, eps2=eps2)

# # ---------------------------------------------------------------
# # Pairwise Mann–Whitney + Cliff’s δ
# # ---------------------------------------------------------------
# def pairwise_posthoc(df_samp, label_col):
#     pairs = list(itertools.combinations(df_samp[label_col].unique(), 2))
#     # wheeze vs normal, crackle vs normal, both vs normal, wheeze vs crackle, wheeze vs both, crackle vs both
#     out = []
#     for a, b in pairs:
#         x = df_samp.loc[df_samp[label_col]==a, "CycleDur"].values
#         y = df_samp.loc[df_samp[label_col]==b, "CycleDur"].values
#         # Test for difference in distribution using Mann–Whitney U test
#         # Assumes that x and y are independent samples and non-normally distributed 
#         stat, p = mannwhitneyu(x, y, alternative="two-sided")
#         # Cliff's delta quantifies the effect size
#         delta = cliffs_delta(x, y)
#         out.append({"pair": f"{a} vs {b}", "p": p, "cliff": delta})
#     df_out = pd.DataFrame(out)
#     df_out["p_adj"] = np.minimum(df_out["p"] * len(df_out), 1.0)
#     return df_out
def common_language_effect(x, y):
    nx, ny = len(x), len(y)
    diff = np.subtract.outer(x, y)
    prob = np.sum(diff > 0) / (nx * ny)
    return prob

def hodges_lehmann(x, y):
    # Median of all pairwise differences (robust typical difference)
    return np.median(np.subtract.outer(x, y))

# ---------------------------------------------------------------
# Pairwise Mann–Whitney + δ + CLES + HL_diff
# ---------------------------------------------------------------
def pairwise_posthoc(df_samp, label_col):
    pairs = list(itertools.combinations(df_samp[label_col].unique(), 2))
    out = []
    for a, b in pairs:
        x = df_samp.loc[df_samp[label_col]==a, "CycleDur"].values
        y = df_samp.loc[df_samp[label_col]==b, "CycleDur"].values
        # Test for difference in distribution using Mann–Whitney U test
        # Different if p < 0.05
        stat, p = mannwhitneyu(x, y, alternative="two-sided")
        # print(f"Pairwise test {a} vs {b}: p={p}")
        delta = cliffs_delta(x, y)
        cles = common_language_effect(x, y)
        hl_diff = hodges_lehmann(x, y)
        out.append({"pair": f"{a} vs {b}", "p": p, "cliff": delta, "cles": cles, "hl_diff": hl_diff})
    df_out = pd.DataFrame(out)
    df_out["p_adj"] = np.minimum(df_out["p"] * len(df_out), 1.0)
    return df_out

# ---------------------------------------------------------------
# Bootstrap routine (valid for 2c or 4c)
# ---------------------------------------------------------------
def bootstrap_cycle_duration(df, label_col="label_4c", n_iter=200, seed=42):
    np.random.seed(seed)
    main_res, posthoc_res = [], []

    for i in range(n_iter):
        # samp = stratified_patient_sampling(df, label_col, seed+i)
        samp = stratified_patient_sampling_priority(df, label_col, seed+i)
        main = cycle_duration_test(samp, label_col)
        post = pairwise_posthoc(samp, label_col)
        post["iter"] = i
        main_res.append(main)
        posthoc_res.append(post)

    df_main = pd.DataFrame(main_res)
    df_post = pd.concat(posthoc_res, ignore_index=True)

    # =============================
    # 🧾  GLOBAL SUMMARY
    # =============================
    print("\n" + "="*90)
    print(f"📊 CYCLE DURATION TEST — {label_col.upper()}".center(90))
    print("="*90)
    print(df_main["test"].value_counts(), "\n")

    mean_p = df_main.p.mean()
    frac_sig = (df_main.p < 0.05).mean()
    mean_eps2 = df_main.loc[df_main.test=="Kruskal_Wallis", "eps2"].mean()

    print(f"Mean p-value: {mean_p:.3f}")
    print(f"Fraction significant (p<0.05): {frac_sig:.2f}")
    print(f"Mean ε² (effect size): {mean_eps2:.4f}")
    print("→ ε² represents the proportion of total variance in cycle duration\n"
          "   explained by the label categories. Values <0.01 = negligible, 0.01–0.06 = small.\n")

    # =============================
    # 🔍  PAIRWISE SUMMARY
    # =============================
    print("="*90)
    print("🔍 PAIRWISE POST-HOC RESULTS (Mann–Whitney + Effect Sizes)".center(90))
    print("="*90)

    summary = (
        df_post.groupby("pair")
        .agg(mean_p=("p_adj","mean"),
             frac_sig=("p_adj", lambda x:(x<0.05).mean()),
             mean_cliff=("cliff","mean"),
             std_cliff=("cliff","std"),
             mean_cles=("cles","mean"),
             std_cles=("cles","std"),
             mean_hl=("hl_diff","mean"),
             std_hl=("hl_diff","std"))
        .reset_index()
        .sort_values("frac_sig", ascending=False)
    )

    for _,r in summary.iterrows():
        strength = (
            "negligible" if abs(r["mean_cliff"]) < 0.147 else
            "small" if abs(r["mean_cliff"]) < 0.33 else
            "medium" if abs(r["mean_cliff"]) < 0.474 else "large"
        )
        print(f"{r['pair']:<18} | sig%={r['frac_sig']*100:5.1f}% | "
              f"δ={r['mean_cliff']:+.3f} ({strength}) | "
              f"CLES={r['mean_cles']:.3f}±{r['std_cles']:.3f} | "
              f"HL_diff={r['mean_hl']:+.3f}s ±{r['std_hl']:.3f}")

    print("\nInterpretation guide:")
    print(" • δ (Cliff’s):  direction & standardized magnitude of difference (−1 to 1)")
    print(" • CLES:         probability that a random sample from A > B (0.5 = no diff)")
    print(" • HL_diff:      median difference in seconds (real-world magnitude)")
    print(" • sig%:         stability of p<0.05 across bootstraps")

    return df_main, df_post, summary

In [26]:
# For 4-class problem
df_main, df_post, summary = bootstrap_cycle_duration(df, label_col="label_4c", n_iter=200)


                             📊 CYCLE DURATION TEST — LABEL_4C                             
test
Kruskal_Wallis    200
Name: count, dtype: int64 

Mean p-value: 0.068
Fraction significant (p<0.05): 0.58
Mean ε² (effect size): 0.0506
→ ε² represents the proportion of total variance in cycle duration
   explained by the label categories. Values <0.01 = negligible, 0.01–0.06 = small.

                🔍 PAIRWISE POST-HOC RESULTS (Mann–Whitney + Effect Sizes)                 
Crackle vs Normal  | sig%= 41.0% | δ=+0.381 (medium) | CLES=0.690±0.042 | HL_diff=+0.760s ±0.200
Both vs Normal     | sig%= 32.5% | δ=+0.335 (medium) | CLES=0.667±0.045 | HL_diff=+0.671s ±0.189
Wheeze vs Normal   | sig%=  2.0% | δ=+0.178 (small) | CLES=0.589±0.047 | HL_diff=+0.365s ±0.208
Both vs Crackle    | sig%=  0.0% | δ=-0.061 (negligible) | CLES=0.469±0.031 | HL_diff=-0.129s ±0.135
Both vs Wheeze     | sig%=  0.0% | δ=+0.132 (negligible) | CLES=0.566±0.031 | HL_diff=+0.269s ±0.135
Wheeze vs Crackle  | sig%=  0.0%

Breathing-cycle duration analysis (independent-patient sampling)
After restricting to one randomly selected cycle per patient to ensure sample independence, we observed a small but consistent association between sound category and breathing-cycle duration (Kruskal–Wallis p ≈ 0.07, ε² = 0.05).
Crackle and Both cycles were typically longer than Normal by ≈ 0.7 s (Cliff’s δ ≈ 0.35–0.38, CLES ≈ 0.68), indicating a 68–70 % probability that a randomly selected Crackle/Both cycle exceeds the duration of a Normal one.
Wheeze cycles were only marginally longer (δ ≈ 0.18) and rarely significant.
Overall, cycle duration explains roughly 5 % of variance across sound types—suggesting that while duration correlates weakly with abnormality, it alone carries limited discriminative power for classification.

In [ ]:
# For 2-class problem (Normal vs Abnormal)
df_main2, df_post2, summary2 = bootstrap_cycle_duration(df, label_col="label_2c", n_iter=200)

In [ ]:
from scipy.stats import kruskal

# Kruskal-Wallis test for CycleDur across 4-class labels
groups = [df.loc[df["label_4c"] == lbl, "CycleDur"] for lbl in df["label_4c"].unique()]

stat, p = kruskal(*groups)
print(f"Kruskal-Wallis H-statistic: {stat:.2f}, p-value: {p:.6f} → {'Reject H₀' if p < 0.05 else 'Fail to reject H₀'}")
if p < 0.05:
    print("Significant differences in breathing cycles distributions across sound labels.")
else:
    print("No significant differences in breathing cycles distributions across sound labels.")

## Follow-up (if significant):

Post-hoc Dunn test
→ Which pairs differ?

In [ ]:
from scikit_posthocs import posthoc_dunn

# Post-hoc Dunn test if Kruskal-Wallis was significant
posthoc = posthoc_dunn(df, val_col="CycleDur", group_col="label_4c", p_adjust="bonferroni")
print("\nPost-hoc Dunn test p-values (Bonferroni corrected):")
print(posthoc.round(4))

In [ ]:
for i in range(posthoc.shape[0]):
    for j in range(i+1, posthoc.shape[1]):
        p = posthoc.iloc[i, j]
        if p < 0.01:
            print(f"Significant difference (p < 0.01) between '{posthoc.index[i]}' and '{posthoc.columns[j]}' (p={p:.4f})")

## Q2. Is cycle duration associated with abnormality presence (2-class)?

H₀:

Mean/median cycle duration is independent of being Normal vs Abnormal.

H₁:

Abnormal cycles (Crackle/Wheeze/Both) have significantly different durations.

In [ ]:
from scipy.stats import mannwhitneyu

normal = df.loc[df["label_2c"]=="Normal", "CycleDur"]
abn = df.loc[df["label_2c"]=="Abnormal", "CycleDur"]

u, p = mannwhitneyu(normal, abn, alternative="two-sided")
print(f"Mann–Whitney U={u:.2f}, p={p:.6f}")

### Duration patterns & split drift

* Means (sec): Normal 2.61, Crackle 2.79, Wheeze 2.70, Both 3.06.
* Abnormal cycles tend to be slightly longer (~2.8–3.0s) than Normal (~2.6s).
* Test Crackle is longer than Train Crackle (3.09s vs 2.62s). Test Normal is shorter (2.39s vs 2.77s).  

    → Different segmentation habits or case mix across splits. Fixed windowing must be chosen carefully.

* Long Normal cycles up to 16.16s in train; maxes for other classes ~7–9s.  
    → A few very long “Normal” cycles may dominate minibatches unless clipped/segmented.

## Digital Stethoscope usage distribution per class

In [ ]:
# Device × label
dev_tab = df.pivot_table(index="Device", columns="label_4c", values="Filename", aggfunc="count").fillna(0).astype(int)
dev_prop = dev_tab.div(dev_tab.sum(axis=1), axis=0).round(3)
# print("\nDevice × 4c counts:\n", dev_tab)
# print("\nDevice × 4c proportions:\n", dev_prop)
print_section("Device × Label Counts", dev_tab)
print_section("Device × Label Proportions", dev_prop)

In [ ]:
# Device × Label × Split counts
device_split_counts = (
    df.groupby(["Split", "Device", "label_4c"])
      .size()
      .reset_index(name="count")
      .pivot_table(index=["Device"], columns=["Split", "label_4c"], values="count", fill_value=0)
      .astype(int)
)

# Normalize within each Split to get proportions
device_split_props = device_split_counts.div(device_split_counts.sum(axis=0), axis=1).round(3)

print_section("DEVICE × LABEL × SPLIT — COUNTS", device_split_counts)
print_section("DEVICE × LABEL × SPLIT — PROPORTIONS", device_split_props)

### Device shift is real — and significant

* LittC2SE is missing from test (all LittC2SE lives in train).
* For test Crackle, ~84% come from AKGC417L; for train Crackle, ~82% AKGC417L too, but Meditron/Litt share differs.
* Meditron is Normal-heavy in train (723 of train Normals) and present in test too, but proportions differ.  
    --> Model can learn device acoustics as a shortcut; zero-shot generalization to LittC2SE (in test) is impossible because it isn’t there.

### Ausculation sites stats

In [ ]:
# Auscultatoin Location × label
loc_tab = df.pivot_table(index="AuscLoc", columns="label_4c", values="Filename", aggfunc="count").fillna(0).astype(int)
loc_prop = loc_tab.div(loc_tab.sum(axis=1), axis=0).round(3)
print_section("Auscultation Site × Label Counts", loc_tab)
print_section("Auscultation Site × Label Proportions", loc_prop)
# print("\nAuscLoc × 4c counts:\n", loc_tab)
# print("\nAuscLoc × 4c proportions:\n", loc_prop)

In [ ]:
# Device × Label × Split counts
sites_split_counts = (
    df.groupby(["Split", "AuscLoc", "label_4c"])
      .size()
      .reset_index(name="count")
      .pivot_table(index=["AuscLoc"], columns=["Split", "label_4c"], values="count", fill_value=0)
      .astype(int)
)

# Normalize within each Split to get proportions
sites_split_props = sites_split_counts.div(sites_split_counts.sum(axis=0), axis=1).round(3)

print_section("Ausc × LABEL × SPLIT — COUNTS", sites_split_counts)
print_section("Ausc × LABEL × SPLIT — PROPORTIONS", sites_split_props)

### Auscultation-site shift also exists

* Trachea (Tc) is plentiful in train (417 Normal etc.) but has no “Both” in test and fewer total; site composition differs across splits.
* Posterior sites (Pl/Lr/Ll) show much more Crackle (≈37–40%): consistent clinically but also induces correlation.

### Risks & likely failure modes
1. Device/site confounding: model learns device or ausc-site timbre cues; appears strong from tables.
2. Distribution drift: train has more abnormal; test has shorter Normal & longer Crackle → calibration/thresholds drift.
3. Class “Both” is smallest (7.3%): confusion with Wheeze likely; mislabels possible.
4. Zero coverage: LittC2SE absent in test → can’t assess generalization to that device; results may look better/worse by accident.
5. Potential leakage history: previously suspected a PID in both splits — keep enforcing patient-wise uniqueness.

### Summary

In [ ]:
# ---- COMPLETE DATASET SUMMARY & INTERPRETATION ----
total_cycles = len(df)
total_patients = df["PID"].nunique()
split_counts = df["Split"].value_counts().to_dict()

print("\n" + "="*90)
print("DATASET COMPOSITION SUMMARY")
print("="*90)
print(f"  - Total cycles: {total_cycles:,}")
print(f"  - Total unique patients: {total_patients}")
print(f"  - Split distribution: {split_counts}")
print(f"  - Normal vs Abnormal (2-class): {df['label_2c'].value_counts(normalize=True).round(3).to_dict()}")
print(f"  - 4-Class composition (Normal, Crackle, Wheeze, Both): "
      f"{df['label_4c'].value_counts(normalize=True).round(3).to_dict()}")
print(f"  - Most frequent abnormal type: {df['label_4c'].value_counts().iloc[1:].idxmax()}")
print(f"  - Rarest class: {df['label_4c'].value_counts().idxmin()} "
      f"(≈{df['label_4c'].value_counts(normalize=True).min():.1%} of all cycles)")
print()

# ---- Split sanity ----
print("="*90)
print("SPLIT BALANCE (Official ICBHI Train/Test)")
print("="*90)
for s, group in by_split_prop_4c.iterrows():
    n = split_counts.get(s, 0)
    print(f"  • {s:<5} ({n} cycles) → "
          f"Normal={group['Normal']:.2f}, Crackle={group['Crackle']:.2f}, "
          f"Wheeze={group['Wheeze']:.2f}, Both={group['Both']:.2f}")
print("    → Test set is more 'Normal'-heavy and has fewer Crackles/Both cycles than Train.")
print("      Expect slightly inflated accuracy but lower sensitivity for Crackle/Both on Test.")
print()

# ---- Duration intuition ----
print("="*90)
print("CYCLE DURATION INSIGHTS")
print("="*90)
for lbl, row in dur_stats.iterrows():
    print(f"  - {lbl:<8}: mean={row['mean']:.2f}s | median={row['50%']:.2f}s | max={row['max']:.2f}s")
print("    → Abnormal cycles (Crackle/Both) are longer (~2.8–3.1s) than Normal (~2.6s).")
print("      Test Crackles (~3.1s) are notably longer than Train Crackles (~2.6s) → potential annotation drift.")
print("      Cycle durations range up to 16s (long Normal cycles), so outlier handling or re-segmentation may be needed.")
print()

# ---- Device bias intuition ----
print("="*90)
print("DEVICE DISTRIBUTION & DOMAIN INSIGHTS")
print("="*90)
device_counts = df["Device"].value_counts().to_dict()
print(f"  - Device counts: {device_counts}")
print("  - AKGC417L dominates (>50% of total cycles).")
print("  - Littmann devices (3200, C2SE) show higher Wheeze proportions (~20%),")
print("    while Meditron contributes mostly Normal cycles (~70%).")
print("  - LittC2SE is found only in Train — absent in Test — creating unseen-domain conditions.")
print("    → High risk of device-domain bias: model could learn device timbre instead of pathology.")
print()

# Device availability per split
devices_by_split = df.groupby("Split")["Device"].unique()
print("  Devices present per split:")
for s, dlist in devices_by_split.items():
    print(f"   • {s:<6}: {', '.join(sorted(dlist))}")
shared_devices = set(devices_by_split.get("train", [])) & set(devices_by_split.get("test", []))
missing_devices = set(devices_by_split.get("train", [])) ^ set(devices_by_split.get("test", []))
print(f"\n  Shared devices across splits: {', '.join(sorted(shared_devices)) or 'None'}")
print(f"  Devices exclusive to one split: {', '.join(sorted(missing_devices)) or 'None'}")
print("    → Domain generalization testing limited for devices missing in one subset.")
print()

# ---- Auscultation site intuition ----
print("="*90)
print("AUSCULTATION SITE DISTRIBUTION")
print("="*90)
site_counts = df["AuscLoc"].value_counts().to_dict()
print(f"  - Site counts: {site_counts}")
print("  - Posterior (Pl, Pr, Ll, Lr) sites exhibit higher Crackle proportions (37–40%).")
print("  - Trachea (Tc) is largely Normal (~70%), physiologically expected.")
print("  - Anterior sites (Al, Ar) show balanced distribution but fewer Both cycles.")
print("    → Indicates strong anatomical dependency; advisable to include site metadata or stratify by site.")
print()

# ---- Quick domain-level intuition ----
print("="*90)
print("DOMAIN & DISTRIBUTION INSIGHTS SUMMARY")
print("="*90)
print("• Train/Test label distribution differs mildly → potential class prevalence bias.")
print("• Duration shift (Train shorter Normals, Test longer Crackles) suggests segmentation bias.")
print("• Device shift: LittC2SE appears only in Train, others unbalanced across splits.")
print("• Site shift: Posterior sites dominate Crackle/Both, Trachea largely Normal.")
print("• Combined effect → multi-source domain shift (device + site).")
print("• Recommendation: enforce patient-wise stratified folds preserving Device & Site proportions.")
print("  and evaluate per-device & per-site performance for fairness and robustness.")
print("="*90)

## Deep Metadata Diagnostics

This will go beyond plain EDA to quantify, visualize, and diagnose confounds using statistical tests, correlations, and domain-overlap metrics.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import chi2_contingency, entropy
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

In [ ]:
sns.set(style="whitegrid", font_scale=1.1)

In [ ]:
# Chi-square tests — independence between categorical factors

def chi_square_test(df, var1, var2):
    tab = pd.crosstab(df[var1], df[var2])
    chi2, p, dof, _ = chi2_contingency(tab)
    print(f"Chi-square test for {var1} ↔ {var2}")
    print(f"  χ²={chi2:.2f}, dof={dof}, p={p:.4f}")
    if p < 0.05:
        print("Significant association (non-random relationship)\n")
    else:
        print("Likely independent\n")

# Core dependencies
chi_square_test(df, "Device", "label_4c")
chi_square_test(df, "AuscLoc", "label_4c")
chi_square_test(df, "Split", "label_4c")
chi_square_test(df, "Device", "AuscLoc")

1. Device ↔ Label χ² = 622 (p ≪ 0.001) Devices capture distinct label distributions; risk of spectral bias.
2. Site ↔ Label χ² = 407 (p ≪ 0.001) Strong anatomical correlation (expected biologically, but must control).  
3. Split ↔ Label χ² = 71 (p ≪ 0.001) The “test” subset has class skew — may inflate accuracy.  
4. Device ↔ Site χ² = 330 (p ≪ 0.001) Devices often tied to specific auscultation sites → compounding bias 


All metadata factors are statistically coupled with labels, indicating domain bias.

In [ ]:
chi_results = {
    "Device ↔ Label_4c": {"chi2": 622.31, "dof": 9, "p": 0.0000},
    "AuscLoc ↔ Label_4c": {"chi2": 406.60, "dof": 18, "p": 0.0000},
    "Split ↔ Label_4c": {"chi2": 71.42, "dof": 3, "p": 0.0000},
    "Device ↔ AuscLoc": {"chi2": 330.06, "dof": 18, "p": 0.0000},
}

In [ ]:
#  Joint distributions – multi-dimensional cross tabs
cross = pd.crosstab([df["Device"], df["AuscLoc"]], df["label_4c"], normalize="index")
print("\nDevice × Auscultation × Label (proportion):")

plt.figure(figsize=(10,6))
sns.heatmap(cross, cmap="coolwarm", annot=False)
plt.title("Device × AuscLoc × Label — Proportional Distribution")
plt.tight_layout()
plt.show()

In [ ]:
df.groupby("Filename")["label_4c"].value_counts(normalize=True).unstack(fill_value=0)

In [ ]:
# Patient & recording-level consistency
rec_summary = df.groupby("Filename")["label_4c"].value_counts(normalize=True).unstack(fill_value=0)
rec_summary["entropy"] = rec_summary.apply(lambda x: entropy(x + 1e-8), axis=1)
print("\nRecording-level label diversity (entropy):")
print(rec_summary["entropy"].describe().round(3))

entropy_mean = rec_summary["entropy"].mean()
entropy_pure = (rec_summary["entropy"] == 0).mean()

sns.histplot(rec_summary["entropy"], bins=30)
plt.title("Recording-level Label Diversity (Entropy)")
plt.xlabel("Entropy (0=pure, >0=mixed)")
plt.show()

### “Recording-level entropy mean ≈ 0.35”

→ Half of recordings contain mixed labels (Normal + Abnormal cycles).  
→ Indicates intra-patient variability — realistic but means sequence models might see label noise inside single files.

In [ ]:
# Duration patterns per Device / Site / Label
plt.figure(figsize=(10,6))
sns.boxplot(data=df, x="Device", y="CycleDur", hue="label_4c")
plt.title("Cycle Duration Distribution by Device and Label")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,6))
sns.boxplot(data=df, x="AuscLoc", y="CycleDur", hue="label_4c")
plt.title("Cycle Duration Distribution by AuscLoc and Label")
plt.tight_layout()
plt.show()

In [ ]:
# Domain coverage & blind spots
coverage = df.groupby(["Device", "AuscLoc", "label_4c"]).size().unstack(fill_value=0)
missing = (coverage == 0).sum(axis=1)
missing_sites = (coverage == 0).sum(axis=1).sum()

plt.figure(figsize=(9,5))
sns.heatmap((coverage>0).astype(int), cmap="Greens", cbar=False)
plt.title("Coverage Map — Label presence across Device × Site")
plt.show()

In [ ]:
print(f"  - Missing label combinations per (Device, Site): {9}")

### “Missing label combinations per (Device, Site)”

Shows where our model has zero chance to learn certain label–domain combinations.
E.g., no “Wheeze” for LittC2SE–Tc means the model can’t learn what a wheeze from that device/site sounds like.

Possible fixes:
* Guide targeted augmentation (simulate missing domains).  
* Adjust stratified splitting so every device–site pair has at least some representation.

In [ ]:
# Simple leakage check — can labels predict Device?

X = pd.get_dummies(df["label_4c"], drop_first=False)
y = LabelEncoder().fit_transform(df["Device"])
clf = LogisticRegression(max_iter=200)
f1_macro = make_scorer(f1_score, average='macro')
scores_dev_f1 = cross_val_score(clf, X, y, cv=5, scoring=f1_macro)
scores_dev_acc = cross_val_score(clf, X, y, cv=5, scoring="accuracy")

# Predict Label from Metadata
X_meta = pd.get_dummies(df[["Device", "AuscLoc", "Split"]])
y_lbl = LabelEncoder().fit_transform(df["label_4c"])
clf2 = LogisticRegression(max_iter=200)
scores_lbl_f1 = cross_val_score(clf2, X_meta, y_lbl, cv=5, scoring=f1_macro)
scores_lbl_acc = cross_val_score(clf2, X_meta, y_lbl, cv=5, scoring="accuracy")

In [ ]:
# Entropy & KL divergence — train vs test comparison
def kl_divergence(p, q):
    p, q = np.asarray(p)+1e-9, np.asarray(q)+1e-9
    return np.sum(p * np.log(p / q))

train_dist = df[df["Split"]=="train"]["label_4c"].value_counts(normalize=True).reindex(["Normal","Crackle","Wheeze","Both"]).fillna(0)
test_dist  = df[df["Split"]=="test"]["label_4c"].value_counts(normalize=True).reindex(["Normal","Crackle","Wheeze","Both"]).fillna(0)

kl = kl_divergence(train_dist, test_dist)
kl

In [ ]:
# Calcualte leaky patients
dup_pids = df.groupby("PID")["Split"].nunique()
overlap_pids = dup_pids[dup_pids > 1].index.tolist()

In [ ]:
# Helper for colored risk flags
def color_flag(value, thresholds=(0.05, 0.1), inverse=False):
    from termcolor import colored
    """Return colored qualitative flag depending on value."""
    if inverse:  # smaller is worse
        if value < thresholds[0]: return colored("🔴 High", "red")
        if value < thresholds[1]: return colored("🟡 Moderate", "yellow")
        return colored("🟢 Low", "green")
    else:        # larger is worse
        if value > thresholds[1]: return colored("🔴 High", "red")
        if value > thresholds[0]: return colored("🟡 Moderate", "yellow")
        return colored("🟢 Low", "green")

In [ ]:
# ================================================================
# Pretty print diagnostic report
# ================================================================
print("\n" + "="*100)
print("METADATA DEPENDENCY & DOMAIN BIAS REPORT (AUTO)")
print("="*100)

# Patient overlap
print("Patient overlap check:")
if overlap_pids:
    print(f"  🔴 {len(overlap_pids)} patients appear in both splits → leakage risk!")
    print(f"  PIDs: {', '.join(map(str, overlap_pids))}")
else:
    print("  🟢 No patient overlap detected (clean split).")

# Chi-square associations
print("\nVariable Associations (χ² tests):")
for name, vals in chi_results.items():
    p = vals["p"]
    significance = "🔴 Significant" if p < 0.001 else "🟡 Weak"
    print(f"  • {name:<25} χ²={vals['chi2']:>7.2f} (dof={vals['dof']})  p={p:.4f} → {significance}")
print("    → All metadata factors are statistically coupled with labels, indicating domain bias.\n")

# Leakage metrics
print("Metadata predictability tests:")
print(f"  - Predicting Device from Label: acc={scores_dev_acc.mean():.3f}, "
      f"F1-macro={scores_dev_f1.mean():.3f} → {color_flag(scores_dev_acc.mean(), (0.4, 0.5))} confounding")
print(f"  - Predicting Label from Metadata: acc={scores_lbl_acc.mean():.3f}, "
      f"F1-macro={scores_lbl_f1.mean():.3f} → {color_flag(scores_lbl_acc.mean(), (0.35, 0.45))} confounding")
print("    → Metadata alone explains substantial variance in labels — strong device/site correlation.\n")

# KL divergence
print(f"KL Divergence (Train→Test label distributions): {kl:.3f} → {color_flag(kl, (0.02, 0.05))}")
print("    → Indicates mild domain drift between training and test label compositions.\n")

# Coverage & entropy
print("Coverage & diversity:")
print(f"  - Missing label combinations per (Device, Site): {missing_sites}")
print(f"  - Mean recording entropy: {entropy_mean:.2f} (0=pure, >0=mixed)")
print(f"  - % of pure-label recordings: {entropy_pure*100:.1f}%")
print("    → Mixed recordings indicate intra-patient label heterogeneity (multi-label or temporal variability).\n")

# Cycle duration drift summary
dur = df.groupby("label_4c")["CycleDur"].describe()
print("Cycle duration drift:")
for lbl, row in dur.iterrows():
    print(f"  - {lbl:<8}: mean={row['mean']:.2f}s | median={row['50%']:.2f}s | max={row['max']:.2f}s")
print("    → Abnormal cycles (Crackle/Both) slightly longer (~2.8–3.1s).")
print("      Train–test difference in Crackle duration hints at annotation drift.\n")

# Device–site coverage overview
device_counts = df["Device"].value_counts(normalize=True).round(3).to_dict()
print("Device–Site coverage:")
print(f"  - Device proportions: {device_counts}")
print("  - Dominant device: AKGC417L (>50%)")
print("  - LittC2SE absent in test → unseen domain for that hardware.")
print("  - Posterior sites (Pl, Pr, Ll, Lr) more crackle-heavy; Trachea mostly Normal (~70%).\n")

# Final risk summary
print("="*100)
print(" RISK ASSESSMENT SUMMARY")
print("="*100)
risk_summary = [
    ("Device/Site confounding", "High"),
    ("Patient overlap", "High" if overlap_pids else "Low"),
    ("Split drift (KL)", "Moderate" if kl > 0.02 else "Low"),
    ("Intra-patient variability", "Moderate" if entropy_mean > 0.2 else "Low"),
    ("Class balance", "Low")
]
for r, lvl in risk_summary:
    icon = {"High":"🟥","Moderate":"🟧","Low":"🟩"}[lvl]
    print(f"  {icon} {r:<30} {lvl}")

# Recommendations
print("\nACTIONABLE RECOMMENDATIONS:")
print("  1. Rebuild patient-wise split ensuring no overlap (leakage-free).")
print("  2. Stratify folds by Device & AuscLoc to preserve proportions.")
print("  3. During training:")
print("       • Include Device & Site as conditioning or adversarial inputs.")
print("       • Log per-device & per-site F1-macro metrics.")
print("       • Use EQ jitter, SpecAugment, loudness norm for domain robustness.")
print("  4. During evaluation:")
print("       • Report per-domain (device/site) breakdowns.")
print("       • Use macro-F1 and per-class AUROC for fair reporting.")
print("  5. Consider leave-one-device-out CV for true generalization.\n")

print("="*100)
print("END OF METADATA DIAGNOSTIC REPORT")
print("="*100)

In [ ]:
contingency = pd.crosstab(df["AuscLoc"], df["label_4c"])
chi2, p, dof, expected = chi2_contingency(contingency)

residuals = (contingency - expected) / np.sqrt(expected)
residuals_rounded = residuals.round(2)

print(f"Chi² = {chi2:.2f}, p = {p:.5f}")
print("\nStandardized residuals (positive = over-represented):")
print(residuals_rounded)

In [ ]:
from sklearn.metrics import mutual_info_score
def MI(a,b): return mutual_info_score(a,b)

print("Mutual Information (bits):")
print(f"  Device–Label_4c: {MI(df['Device'], df['label_4c']):.3f}")
print(f"  AuscLoc–Label_4c: {MI(df['AuscLoc'], df['label_4c']):.3f}")
print(f"  Split–Label_4c: {MI(df['Split'], df['label_4c']):.3f}")
print(f"  Device–AuscLoc: {MI(df['Device'], df['AuscLoc']):.3f}")

Global null hypothesis. 

H₀ (global): The distribution of label_4c is independent of auscultation site.  
H₁ (global): The distribution of label_4c differs across auscultation sites.  

→ Test: Chi-square test of independence. 
→ If p < 0.05 → reject H₀ → there is a site–label association.  

For each site s and class c:

H₀(s,c): The probability of observing class c at site s is equal to the overall probability of class c in the dataset.  
H₁(s,c): The probability of observing class c at site s differs from the overall class proportion.  

→ Test: Binomial proportion test or standardized residuals from χ² (which are equivalent asymptotically).  
→ Bonferroni or FDR-corrected p-values handle multiple comparisons.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, norm
from statsmodels.stats.multitest import multipletests

# Contingency table
ct = pd.crosstab(df["AuscLoc"], df["label_4c"])

# 1. Global test
chi2, p_global, dof, expected = chi2_contingency(ct)
print(f"Global χ² = {chi2:.2f}, p = {p_global:.6f} → {'Reject H₀' if p_global < 0.05 else 'Fail to reject H₀'}")

# 2. Post-hoc residual analysis
residuals = (ct - expected) / np.sqrt(expected)
z_scores = residuals.values.flatten()
pvals = 2 * (1 - norm.cdf(np.abs(z_scores)))  # two-tailed p-values
pvals_corrected = multipletests(pvals, method="fdr_bh")[1]  # FDR correction

# Put results back in dataframe
posthoc_df = pd.DataFrame({
    "AuscLoc": np.repeat(ct.index, ct.shape[1]),
    "Label": np.tile(ct.columns, len(ct.index)),
    "z": z_scores,
    "p_uncorrected": pvals,
    "p_corrected": pvals_corrected
})
posthoc_df["signif"] = posthoc_df["p_corrected"] < 0.05
display(posthoc_df.sort_values("p_corrected"))

You can interpret each (site, class) pair like this:  
* z > +2, p < 0.05 → class over-represented at that site.  
* z < −2, p < 0.05 → class under-represented at that site. 

This yields an anatomical map of where each abnormality tends to appear

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

z_mat = residuals.copy()
plt.figure(figsize=(8,4))
sns.heatmap(z_mat, annot=True, center=0, cmap="coolwarm", fmt=".1f")
plt.title("Standardized Residuals (Z-scores) — AuscLoc × Label")
plt.show()

In [ ]:
summary = (
    posthoc_df.query("signif")
    .groupby("AuscLoc")["Label"]
    .apply(list)
    .reset_index()
)
summary.columns = ["AuscLoc", "SignificantOverrepresentedLabels"]
display(summary)

In [ ]:
from scipy.stats import kruskal

# Kruskal-Wallis test for CycleDur across 4-class labels
groups = [df.loc[df["label_4c"] == lbl, "CycleDur"] for lbl in df["label_4c"].unique()]

stat, p = kruskal(*groups)
print(f"Kruskal-Wallis H-statistic: {stat:.2f}, p-value: {p:.6f} → {'Reject H₀' if p < 0.05 else 'Fail to reject H₀'}")
if p < 0.05:
    print("Significant differences in CycleDur distributions across labels.")
else:
    print("No significant differences in CycleDur distributions across labels.")

In [ ]:
from scikit_posthocs import posthoc_dunn

# Perform pairwise Dunn test with Bonferroni correction
posthoc = posthoc_dunn(df, val_col="CycleDur", group_col="label_4c", p_adjust="bonferroni")
print(posthoc.round(4))

In [ ]:
corr = df.assign(
    Sex=(df["Sex"]=="M").astype(int),
    Split=(df["Split"]=="train").astype(int),
).select_dtypes("number").corr()

plt.figure(figsize=(16,6))
sns.heatmap(corr, annot=True, cmap="coolwarm", center=0)
plt.title("Metadata feature correlation matrix")

### No significant correlations found instead the trivial ones

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices

In [ ]:
# ------------------------------------------------
# Preprocessing
# ------------------------------------------------
df_demo = df.copy()
df_demo = df_demo[df_demo["Age"].notna()]
df_demo = df_demo[df_demo["BMI"].notna()]
df_demo["Sex"] = df_demo["Sex"].astype(str)

print("="*90)
print(" DEMOGRAPHIC BIAS ANALYSIS (Age, Sex, BMI)".center(90))
print("="*90, "\n")

# ================================================================
# AGE vs LABEL
# ================================================================
groups = [df_demo.loc[df_demo["label_4c"] == l, "Age"] for l in df_demo["label_4c"].unique()]
stat, p = kruskal(*groups)
print("AGE vs LABEL")
print(f"  Kruskal–Wallis H-statistic: {stat:.2f}, p-value: {p:.6f}")
if p < 0.05:
    print("  🔴 Reject H₀ → Significant age differences across labels.")
else:
    print("  🟢 Fail to reject H₀ → No significant age differences.\n")

# Post-hoc Dunn test
posthoc_age = posthoc_dunn(df_demo, val_col="Age", group_col="label_4c", p_adjust="bonferroni")
print("  Post-hoc Dunn’s test (adjusted p-values):")
print("if p < 0.05, significant difference between groups")
print(posthoc_age.round(4))
print()

# Visualization
sns.boxplot(data=df_demo, x="label_4c", y="Age", palette="Set2")
plt.title("Age distribution across 4-class labels")
plt.show()

# ================================================================
# SEX vs LABEL
# ================================================================
ct_sex = pd.crosstab(df_demo["Sex"], df_demo["label_4c"])
chi2, p, dof, exp = chi2_contingency(ct_sex)
print("="*90)
print(" SEX vs LABEL")
print("="*90)
print(f"  χ² = {chi2:.2f}, p-value = {p:.6f}")
if p < 0.05:
    print("  🔴 Reject H₀ → Sex and label are dependent (imbalanced gender distribution across labels).")
else:
    print("  🟢 Fail to reject H₀ → No significant sex imbalance.\n")

print("\n  Sex × Label contingency table:")
display(ct_sex)

sns.heatmap(ct_sex, annot=True, fmt="d", cmap="YlGnBu")
plt.title("Sex × Label counts")
plt.show()

# ================================================================
# BMI vs LABEL
# ================================================================
groups_bmi = [df_demo.loc[df_demo["label_4c"] == l, "BMI"] for l in df_demo["label_4c"].unique()]
stat, p = kruskal(*groups_bmi)
print("="*90)
print("  BMI vs LABEL")
print("="*90)
print(f"  Kruskal–Wallis H-statistic: {stat:.2f}, p-value: {p:.6f}")
if p < 0.05:
    print("  🔴 Reject H₀ → Significant BMI differences across labels.")
else:
    print("  🟢 Fail to reject H₀ → No significant BMI differences.\n")

# Post-hoc Dunn test for BMI
posthoc_bmi = posthoc_dunn(df_demo, val_col="BMI", group_col="label_4c", p_adjust="bonferroni")
print("  Post-hoc Dunn’s test (adjusted p-values):")
print(posthoc_bmi.round(4))
print()

# Visualization
sns.boxplot(data=df_demo, x="label_4c", y="BMI", palette="Set2")
plt.title("BMI distribution across 4-class labels")
plt.show()

# ================================================================
# LOGISTIC MODEL — Combined demographic predictors
# ================================================================
print("="*90)
print(" Logistic model — Predicting abnormality (Normal vs Abnormal) from demographics")
print("="*90)

df_demo["is_abnormal"] = (df_demo["label_2c"] == "Abnormal").astype(int)
y, X = dmatrices("is_abnormal ~ Age + BMI + C(Sex)", data=df_demo, return_type="dataframe")

model = sm.Logit(y, X).fit(disp=False)
odds_ratios = np.exp(model.params).round(3)
print(model.summary())
print("\n  Odds Ratios:")
print(odds_ratios)
print()

In [ ]:
# ================================================================
# 🧭 AUTOMATED INTERPRETIVE SUMMARY
# ================================================================
print("="*90)
print("📖 DEMOGRAPHIC BIAS — INTERPRETIVE SUMMARY".center(90))
print("="*90)

summary_lines = []

# 1. Age effect
if model.pvalues["Age"] < 0.05:
    if model.params["Age"] > 0:
        summary_lines.append(f"• 🔺 Age is a significant positive predictor (p={model.pvalues['Age']:.3e}) — "
                             f"older patients are more likely to exhibit abnormal sounds "
                             f"(OR={odds_ratios['Age']:.3f}).")
    else:
        summary_lines.append(f"• 🔻 Age is a significant negative predictor — "
                             f"younger patients are more likely abnormal (OR={odds_ratios['Age']:.3f}).")
else:
    summary_lines.append("• 🟢 No significant age effect detected (p≥0.05).")

# 2. BMI effect
if model.pvalues["BMI"] < 0.05:
    if model.params["BMI"] > 0:
        summary_lines.append(f"• ⚖️ Higher BMI increases abnormality likelihood (OR={odds_ratios['BMI']:.3f}).")
    else:
        summary_lines.append(f"• ⚖️ Higher BMI decreases abnormality likelihood (p={model.pvalues['BMI']:.3e}) — "
                             f"leaner individuals show more abnormal cycles (OR={odds_ratios['BMI']:.3f}).")
else:
    summary_lines.append("• 🟢 No significant BMI effect detected.")

# 3. Sex effect
sex_p = model.pvalues.get("C(Sex)[T.M]", np.nan)
sex_coef = model.params.get("C(Sex)[T.M]", 0)
if sex_p < 0.05:
    if sex_coef > 0:
        summary_lines.append(f"• ⚧ Males more likely to produce abnormal cycles (OR={odds_ratios['C(Sex)[T.M]']:.3f}).")
    else:
        summary_lines.append(f"• ⚧ Females more likely to produce abnormal cycles (OR={odds_ratios['C(Sex)[T.M]']:.3f}).")
else:
    summary_lines.append("• 🟢 No significant sex difference observed (p≥0.05).")

# 4. Model goodness
r2 = model.prsquared
llr_p = model.llr_pvalue
summary_lines.append(f"• Model pseudo-R²={r2:.3f}, global p={llr_p:.3e} → overall demographic effects are statistically significant.")

# Print intuitive conclusions
for line in summary_lines:
    print(" ", line)

print("\n📌 Suggested interpretation:")
if (model.pvalues["Age"] < 0.05) and (model.params["Age"] > 0):
    print("   → Abnormal sounds are more frequent among older individuals, possibly reflecting age-related respiratory changes.")
if (model.pvalues["BMI"] < 0.05) and (model.params["BMI"] < 0):
    print("   → Leaner subjects may exhibit clearer crackle/wheeze patterns due to thinner chest walls (less attenuation).")
if sex_p >= 0.05:
    print("   → No gender-driven acoustic differences detected; male/female balance is adequate.")
print("   → Demographic effects exist but explain modest variance (pseudo-R²≈0.04). Consider age/BMI-aware sampling or include demographics as auxiliary inputs.\n")